In [124]:
import pickle

import geopandas as gpd
from shapely import Point, LineString, MultiLineString, Polygon, MultiPolygon
import osmnx as ox
import networkx as nx
import matplotlib
from scipy.spatial import cKDTree
import numpy as np
import networkx as nx

from my_paths import *

In [109]:
PATH_TEST_STAGING = "../Data/Staging/Analisi/"

In [110]:
CRS_GRAD = "EPSG:4326"  # Lat-Long
CRS_METR = "EPSG:32632" # Metri

## Creazione strade da openstreet maps (Da passare su notebook ETL)

Ingestion da openstreet maps

In [111]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)

Conversione nei 2 geodataframe - conversione dei nodi nei due sistemi crs

In [113]:
G_ox_strade_copy = G_ox_strade.copy()
gdf_nodes_METR, gdf_edges = ox.graph_to_gdfs(G_ox_strade_copy)
gdf_nodes_METR = gdf_nodes_METR.to_crs(CRS_METR)
gdf_nodes_GRAD = gdf_nodes_METR.to_crs(CRS_GRAD)

Aggiungiamo archi di congiunzione artificiali per connettere strade molto vicine tra di loro

In [114]:
coords = np.array([[geom.x, geom.y] for geom in gdf_nodes_METR.geometry])

tree = cKDTree(coords)
pairs = tree.query_pairs(r=5)  # distanza massima in metri

for i, j in pairs:
    u = gdf_nodes_GRAD.iloc[i].geometry.coords[0]
    v = gdf_nodes_GRAD.iloc[j].geometry.coords[0]

    if not G_ox_strade_copy.has_edge(u, v):
        length = LineString([gdf_nodes_METR.iloc[i].geometry.coords[0], gdf_nodes_METR.iloc[j].geometry.coords[0]]).length
        G_ox_strade_copy.add_edge(u, v, geometry=LineString([u,v]), artificial=True, weight=length)


Graph -> GeoDataFrame

In [115]:
G_nx = nx.Graph(G_ox_strade_copy.to_undirected())
edge_data = []

for u, v, data in G_nx.edges(data=True):
    data_copy = data.copy()
    data_copy.update({'u': u, 'v': v,})
    edge_data.append(data_copy)

gdf_edges = gpd.GeoDataFrame(edge_data, geometry='geometry', crs=CRS_GRAD)
gdf_edges["osmid"] = gdf_edges["osmid"].apply(lambda x: x[0] if isinstance(x, list) else x)
gdf_edges.loc[gdf_edges["osmid"].isna(), "osmid"] = -1
gdf_edges["osmid"] = gdf_edges["osmid"].astype(int)

Salva GeoJson & Pickle

In [116]:
gdf_edges.to_file(PATH_TEST_STAGING+"strade.geojson", driver="GeoJSON")

with open(PATH_TEST_STAGING+"strade.pickle", "wb") as f:
    pickle.dump(G_nx, f)

## Converte usando networkx (Da passare su notebook ETL)

Dichiariamo 2 funzioni:
1. Conversione: **GeoDataFrame -> Graph**, per quanto riguarda percorsi di Linestring.
2. Conversione: **Graph -> GeoDataFrame**, questa in teoria funziona su tutti i Graph.

In [ ]:
def percorsi_geojson_to_graph(gdf: gpd.GeoDataFrame) -> nx.Graph:
    G = nx.Graph()

    for _, row in gdf.iterrows():
        geom = row.geometry

        coords = list(geom.coords)

        for i in range(len(coords) - 1):
            u = tuple(coords[i])
            v = tuple(coords[i + 1])
            length = gpd.GeoDataFrame([LineString([u, v])], columns=["geometry"], crs=CRS_GRAD).to_crs(CRS_METR)["geometry"][0].length
            G.add_edge(u, v, weight=length, geometry=LineString([u, v]))
    return G

def graph_to_geodataframe(G: nx.Graph) -> gpd.GeoDataFrame:
    edge_data = []
    for u, v, data in G.edges(data=True):
        data_copy = data.copy()
        data_copy.update({'u': u, 'v': v,})
        edge_data.append(data_copy)

    return gpd.GeoDataFrame(edge_data, geometry='geometry', crs=CRS_GRAD)

Modifico ad hoc il raw delle ciclabili. Se decideremo di utilizzare questo metodo dei graph, modificheremo l'ETL delle ciclabili in questo modo e aggiungeremo la conversione in graph e il salvataggio in pickle

In [ ]:
# Carico Raw
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo Multilinestring in Linestring
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])

# Altre operazioni di pulizia. Da considerare di rimuover il group_by dall'ETL delle piste
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

G_ciclabili = percorsi_geojson_to_graph(gdf_ciclabili_staging)
with open(PATH_TEST_STAGING+"rete_ciclabile.pickle", "wb") as f:
    pickle.dump(G_ciclabili, f)

## Proviamo a creare connessioni con networkx

Carichiamo da file i Graph (dai file pickle)

In [ ]:
with open(PATH_TEST_STAGING+"rete_ciclabile.pickle", "rb") as f:
    G_ciclabili = pickle.load(f)
with open(PATH_TEST_STAGING+"strade.pickle", "rb") as f:
    G_strade = pickle.load(f)